In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation

2023-03-23 15:25:14.781973: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('meet_or_beat.csv')
df.head()

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns
0,2.01,1.67,11.0,0.051444,0.018585
1,0.17,0.19,6.0,0.112955,-0.000510
2,-0.07,0.14,4.0,0.077167,-0.046104
3,0.48,0.51,8.0,-0.006130,-0.004899
4,-0.24,-0.27,9.0,0.089762,-0.025466


In [4]:
df.shape

(71968, 5)

In [5]:
# Generate the categorical outcome variable
df['earnings_outcome'] = np.nan
df.loc[(df['EPS']==df['forecasted_eps']), 'earnings_outcome'] = 'meet'
df.loc[(df['EPS']>df['forecasted_eps']), 'earnings_outcome'] = 'beat'
df.loc[(df['EPS']<df['forecasted_eps']), 'earnings_outcome'] = 'lose'
df.head()

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns,earnings_outcome
0,2.01,1.67,11.0,0.051444,0.018585,beat
1,0.17,0.19,6.0,0.112955,-0.000510,lose
2,-0.07,0.14,4.0,0.077167,-0.046104,lose
3,0.48,0.51,8.0,-0.006130,-0.004899,lose
4,-0.24,-0.27,9.0,0.089762,-0.025466,beat


In [6]:
# Preview the output variable 
y = df['earnings_outcome']
y.values

array(['beat', 'lose', 'lose', ..., 'lose', 'beat', 'meet'], dtype=object)

In [7]:
# Encode earnings labels to integers
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y

array([0, 1, 1, ..., 1, 0, 2])

In [8]:
# Save the unique number of labels for future use
number_of_classes = len(list(y.drop_duplicates()))
number_of_classes

3

In [9]:
# Convert labeled integers to a Keras `categorical` data type
from keras.utils.np_utils import to_categorical
y_categorical = to_categorical(encoded_y, num_classes=number_of_classes)
y_categorical

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [10]:
# above 0 maps to beat, 1 to lose, 2 to meet

In [12]:
# Specify X (predictor) variables
X = df[['forecasted_eps',
              'before_total_returns','noOfEsts']]
X.head(3)

,forecasted_eps,before_total_returns,noOfEsts
0,1.67,0.018585,11.0
1,0.19,-0.000510,6.0
2,0.14,-0.046104,4.0


In [13]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state=1)


In [16]:
# Preview the encoded data we're trying to predict
pd.DataFrame(y_train).head(5)

,0,1,2
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,1.0,0.0,0.0


In [17]:
# Check for class balance
pd.DataFrame(y_train).sum()

0    32924.0
1    16878.0
2     4174.0
dtype: float32

In [19]:
# we have a big class imbalance!!!

In [18]:
# Save the count of unique predictor variables for use in model
number_of_predictors = len(X.columns)
number_of_predictors

3

In [20]:
# Build the neural network layers 
model = Sequential()
model.add(Dense(9, input_dim=number_of_predictors, activation='relu'))
model.add(Dense(6, activation='relu'))

2023-03-23 15:49:06.761913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
# Add the final output layer
# With the # of possible outputs equal to the number classes 
model.add(Dense(number_of_classes, activation='softmax'))

In [22]:
# Compile the model (with multi-class specific parameters)
model.compile(loss="categorical_crossentropy", 
              optimizer= "adam", 
              metrics=['categorical_accuracy'])

In [23]:
# Summarise the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 36        
                                                                 
 dense_1 (Dense)             (None, 6)                 60        
                                                                 
 dense_2 (Dense)             (None, 3)                 21        
                                                                 
Total params: 117
Trainable params: 117
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Fit the model
model.fit(X_train,y_train, 
                    epochs=20,
                    batch_size=1000,
                    shuffle=True)

Epoch 1/20
54/54 [==============================] - 1s 1ms/step - loss: 1.5702 - categorical_accuracy: 0.1020
Epoch 2/20
54/54 [==============================] - 0s 997us/step - loss: 1.0397 - categorical_accuracy: 0.4962
Epoch 3/20
54/54 [==============================] - 0s 1ms/step - loss: 0.9107 - categorical_accuracy: 0.6061
Epoch 4/20
54/54 [==============================] - 0s 1ms/step - loss: 0.8905 - categorical_accuracy: 0.6090
Epoch 5/20
54/54 [==============================] - 0s 1ms/step - loss: 0.8825 - categorical_accuracy: 0.6095
Epoch 6/20
54/54 [==============================] - 0s 1ms/step - loss: 0.8768 - categorical_accuracy: 0.6098
Epoch 7/20
54/54 [==============================] - 0s 1ms/step - loss: 0.8723 - categorical_accuracy: 0.6100
Epoch 8/20
54/54 [==============================] - 0s 1ms/step - loss: 0.8689 - categorical_accuracy: 0.6100
Epoch 9/20
54/54 [==============================] - 0s 942us/step - loss: 0.8656 - categorical_accuracy: 0.6100
Epoch 

In [25]:
# Evaluate model on the test data
model.evaluate(X_test,y_test, verbose=2)

563/563 - 0s - loss: 0.8413 - categorical_accuracy: 0.6113 - 437ms/epoch - 775us/step


[0.8412684798240662, 0.6112716794013977]

In [26]:
# Save predictions on the test data
# this displays probabilities for each possible outcome for each row
predictions = model.predict(X_test)
predictions

563/563 [==============================] - 0s 586us/step


array([[0.5796911 , 0.31852967, 0.1017793 ],
       [0.7820684 , 0.16301823, 0.05491337],
       [0.57874095, 0.34979826, 0.07146071],
       ...,
       [0.6419835 , 0.2769918 , 0.08102465],
       [0.57769614, 0.33771816, 0.08458575],
       [0.5279991 , 0.39959583, 0.072405  ]], dtype=float32)

In [27]:
# Get the most likely prediction for each observation
# argmax takes the highest value in each row of the above predictions
most_likely = np.argmax(predictions, axis=1)
most_likely

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
# Evaluate prediction balance
pd.DataFrame(most_likely).value_counts()

0    17440
1      552
dtype: int64